In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv("C:\\Users\\RABIN RESIN\\Desktop\\luminar\\development-journey\\mechine learning journey\\exam\\stroke.csv.xls")
df.shape

(5110, 12)

In [3]:
df[df.duplicated()]

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke


In [4]:
df.isnull().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [5]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [6]:
df.drop(columns="id",inplace=True)

In [7]:
df["stroke"].value_counts()

stroke
0    4861
1     249
Name: count, dtype: int64

In [8]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 439.3+ KB


In [10]:
from sklearn.impute import SimpleImputer
impute=SimpleImputer(missing_values=np.nan,strategy="mean")
x.iloc[:,8:9]=impute.fit_transform(x.iloc[:,8:9])


In [11]:
x.isnull().sum()

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
dtype: int64

In [12]:
from sklearn.preprocessing  import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct=ColumnTransformer([("encoder",OneHotEncoder(),["gender","ever_married","work_type","Residence_type","smoking_status"])],remainder="passthrough")
x=pd.DataFrame(ct.fit_transform(x))

In [13]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
xtrain=scaler.fit_transform(xtrain)
xtest=scaler.transform(xtest)

In [15]:
# smote xtrain ytrain
! pip install imblearn


[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from imblearn.over_sampling import SMOTE
sm =SMOTE(random_state=42)
xtrain_smote,ytrain_smote=sm.fit_resample(xtrain,ytrain)

In [17]:
xtrain

array([[-1.19299477,  1.19299477,  0.        , ..., -0.23618875,
         0.13559307, -0.05801609],
       [ 0.83822664, -0.83822664,  0.        , ..., -0.23618875,
        -0.39745713,  0.94746229],
       [ 0.83822664, -0.83822664,  0.        , ..., -0.23618875,
        -1.02870079,  0.61230283],
       ...,
       [ 0.83822664, -0.83822664,  0.        , ..., -0.23618875,
        -0.20107021, -1.43732618],
       [-1.19299477,  1.19299477,  0.        , ..., -0.23618875,
         1.96444623,  0.26425262],
       [ 0.83822664, -0.83822664,  0.        , ..., -0.23618875,
        -0.13816503,  2.75216707]], shape=(4088, 21))

In [18]:
xtrain_smote


array([[-1.19299477,  1.19299477,  0.        , ..., -0.23618875,
         0.13559307, -0.05801609],
       [ 0.83822664, -0.83822664,  0.        , ..., -0.23618875,
        -0.39745713,  0.94746229],
       [ 0.83822664, -0.83822664,  0.        , ..., -0.23618875,
        -1.02870079,  0.61230283],
       ...,
       [ 0.83822664, -0.83822664,  0.        , ..., -0.23618875,
         0.1811307 , -1.08329897],
       [-0.67942603,  0.67942603,  0.        , ..., -0.23618875,
        -0.4438956 , -0.24828997],
       [-1.19299477,  1.19299477,  0.        , ..., -0.23618875,
         0.09700773,  1.62867057]], shape=(7802, 21))

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,ConfusionMatrixDisplay,precision_score,recall_score,f1_score
models=[KNeighborsClassifier,GaussianNB,DecisionTreeClassifier]
for model in models:
    md=model()
    md.fit(xtrain_smote,ytrain_smote)
    ypred=md.predict(xtest)
    print("========",model,"==========")
    print("accuracy",accuracy_score(ytest,ypred)*100)
    print("f1 score",f1_score(ytest,ypred)*100)

======== <class 'sklearn.neighbors._classification.KNeighborsClassifier'> ==========
accuracy 81.99608610567515
f1 score 17.857142857142858
======== <class 'sklearn.naive_bayes.GaussianNB'> ==========
accuracy 35.0293542074364
f1 score 15.736040609137056
======== <class 'sklearn.tree._classes.DecisionTreeClassifier'> ==========
accuracy 86.88845401174167
f1 score 19.27710843373494


In [20]:
knn=KNeighborsClassifier()
knn.fit(xtrain_smote,ytrain_smote)
ypred=knn.predict(xtest)


In [21]:
parms={
    "n_neighbors":[3,5,7,9],
    "weights":["distance","uniform"],
    "algorithm":["auto","ball_tree","kd_tree","brute"]
}

from sklearn.model_selection import GridSearchCV
gs=GridSearchCV(knn,parms,cv=10)
gs.fit(xtrain_smote,ytrain_smote)


GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [3, 5, 7, 9],
                         'weights': ['distance', 'uniform']})

In [22]:
gs.best_params_

{'algorithm': 'auto', 'n_neighbors': 3, 'weights': 'distance'}

In [23]:
knn=KNeighborsClassifier(n_neighbors=3,algorithm="auto",weights="distance")
knn.fit(xtrain,ytrain)
ypred=knn.predict(xtest)


In [24]:
print(accuracy_score(ytest,ypred)*100)
print(f1_score(ytest,ypred)*100)

92.75929549902152
7.5


In [25]:
dt=DecisionTreeClassifier()

In [26]:
parms1={
    "criterion":["gini", "entropy", "log_loss"],
    "splitter":["best", "random"],
    "max_depth":[3,4,5]
}
gs=GridSearchCV(dt,parms1,cv=10)
gs.fit(xtrain_smote,ytrain_smote)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [3, 4, 5],
                         'splitter': ['best', 'random']})

In [27]:
gs.best_params_

{'criterion': 'gini', 'max_depth': 5, 'splitter': 'best'}

In [28]:
dt=DecisionTreeClassifier(criterion="gini",max_depth=5,splitter="best")
dt.fit(xtrain_smote,ytrain_smote)
ypred=dt.predict(xtest)


In [29]:
print(accuracy_score(ytest,ypred)*100)
print(f1_score(ytest,ypred)*100)

76.81017612524462
28.3987915407855


In [ ]:
# it is used to find which parametres are used to get
dt.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': None,
 'splitter': 'best'}